In [6]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
generated_answers_df = pd.read_csv('/content/drive/MyDrive/submit.csv')
generated_answers = generated_answers_df['generated_response'].tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
knowledge_df = pd.read_csv('/content/drive/MyDrive/vihallu-warmup.csv')
knowledge = knowledge_df['context'].tolist()

In [8]:
!pip install -U transformers
!pip install -U torch

In [9]:
!pip install sentence-transformers

In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

sbert_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

def check_entailment_loop(knowledge_list, generated_answer_list, threshold=0.75, output_file="entailment_results.csv"):
    data = []
    for k, a in zip(knowledge_list, generated_answer_list):
        emb_k = sbert_model.encode(k, convert_to_tensor=True)
        emb_a = sbert_model.encode(a, convert_to_tensor=True)

        sim = util.cos_sim(emb_k, emb_a).item()
        is_entailed = sim >= threshold

        data.append({
            "Knowledge": k,
            "Generated Answer": a,
            "Cosine Similarity": sim,
            "Entailed": is_entailed
        })

    # Lưu ra file CSV
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    return df

df_results = check_entailment_loop(knowledge, generated_answers, output_file="entailment_results.csv")
print(df_results.head())


                                           Knowledge  \
0  Theo pháp lệnh Vincennes năm 1374, vương quốc ...   
1  Hệ thống đường biển xuất phát từ các cảng biển...   
2  Năm 1928, Bộ Giao thông khởi thảo kế hoạch côn...   
3  Sự tiến hóa của giới thực vật đã theo xu hướng...   
4  Kể từ những năm 1970, chính phủ cũng thực hiện...   

                                    Generated Answer  Cosine Similarity  \
0  Quyền phản đối được khôi phục bởi Orléans đã g...           0.510341   
1  Theo ngữ cảnh đã cung cấp, những tuyến đường t...           0.663369   
2  Dựa trên thông tin trong ngữ cảnh, độ dài của ...           0.632572   
3  Dựa trên ngữ cảnh cung cấp, nhóm thực vật kém ...           0.655786   
4  Theo ngữ cảnh, Cục Quản lý Dược phẩm và Thực p...           0.561098   

   Entailed  
0     False  
1     False  
2     False  
3     False  
4     False  


In [11]:
entailed_rows = df_results[df_results['Entailed'] == True]
display(entailed_rows)

,Knowledge,Generated Answer,Cosine Similarity,Entailed
7,"Ngày 5 tháng 6 năm 1948, Bảo Đại và Bollaert k...",Nội dung chính của Hiệp ước Vịnh Hạ Long bao g...,0.855858,True
8,Trải qua nhiều thay đổi lớn về địa tầng và địa...,Phong Nha-Kẻ Bàng phô diễn các bằng chứng ấn t...,0.772831,True
10,"Ngày 27 tháng 8, Leopold và Vua Frederick Will...",Thông cáo Pillnitz trong mắt Leopold được coi ...,0.798865,True
12,"Hague bắt nguồn từ năm 1230, khi Count Floris ...","Ngữ cảnh: Hague bắt nguồn từ năm 1230, khi Cou...",0.977334,True
14,Hầu hết những người nhiễm HIV-1 nếu không được...,Đúng. Nếu không điều trị bằng kháng retrovirus...,0.774021,True
...,...,...,...,...
185,"Ngày 21/9/1945, Thiếu tướng Anh Gracey ra bản ...","Đúng, theo ngữ cảnh, Thiếu tướng Anh Gracey đã...",0.878425,True
187,Các chứng cứ đầu tiên về sự xuất hiện của thực...,"Đúng, bạn đã hiểu chính xác. Thực tế, các mẫu ...",0.879992,True
188,"Tuy nghi ngờ, thậm chí thù địch nhau nhưng các...","Ba tổ chức Việt Minh, Việt Quốc và Việt Cách đ...",0.804358,True
195,Các quy định pháp luật về ngân hàng tại Mỹ đượ...,Dịch vụ ngân hàng ở Mỹ có một hệ thống quản lý...,0.875981,True
